#  Capstone Project Assigment The Battle of Neighborhoods


### Import Library

In [1]:
import requests
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium
import json
from IPython.display import Image 
from IPython.core.display import HTML 

# FourSquare Api Credentaials

In [2]:
ClIENT_ID = 'G4G3BSMIJBOKSRFTLMKCCHCOYYAA5B4WMD1AHJPG0YN5RJ5P' # your Foursquare ID
ClIENT_SECRET = 'DSQT5Q4DQVFK04CDBSMY2ALHS5W50FZFVIYTQAZVLXWXKTCR' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: G4G3BSMIJBOKSRFTLMKCCHCOYYAA5B4WMD1AHJPG0YN5RJ5P
Foursquare_Secret:DSQT5Q4DQVFK04CDBSMY2ALHS5W50FZFVIYTQAZVLXWXKTCR


In [3]:
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


# Create DataFerame Hotel

In [4]:
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=G4G3BSMIJBOKSRFTLMKCCHCOYYAA5B4WMD1AHJPG0YN5RJ5P&client_secret=DSQT5Q4DQVFK04CDBSMY2ALHS5W50FZFVIYTQAZVLXWXKTCR&ll=43.653963,-79.387207&v=20180604&query=Hotel&radius=500&limit=30'

In [5]:
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-5-bc0f259b5133>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4ab2d511f964a5209b6c20e3,Sheraton Centre Toronto Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1584887028,False,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",432,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,NaN
1,4b68aed1f964a520de862be3,The Rex Hotel Jazz & Blues Bar,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",v-1584887028,False,194 Queen St W,Queen & St. Patrick,43.650505,-79.388577,"[{'label': 'display', 'lat': 43.65050475544005...",400,M5V 1Z1,CA,Toronto,ON,Canada,"[194 Queen St W (Queen & St. Patrick), Toronto...",62225795,NaN
2,4f343a31e4b0230a3b337a90,VFM Test Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1584887028,False,123 Test Drive,at somewhere St,43.658434,-79.387894,"[{'label': 'display', 'lat': 43.658434, 'lng':...",500,M2M 2M2,CA,Toronto,ON,Canada,"[123 Test Drive (at somewhere St), Toronto ON ...",NaN,NaN
3,5545d07e498e2facac03f666,Sheraton Centre Toronto Hotel - Grand Ballroom,"[{'id': '56aa371be4b08b9a8d5734cf', 'name': 'B...",v-1584887028,False,123 Queen Street West,at Bay Street,43.651200,-79.384520,"[{'label': 'display', 'lat': 43.65119976208405...",376,NaN,CA,Toronto,ON,Canada,"[123 Queen Street West (at Bay Street), Toront...",NaN,Financial District
4,4f53fb2ee4b036244bea152f,Stathcona Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1584887028,False,NaN,NaN,43.654947,-79.386359,"[{'label': 'display', 'lat': 43.654947, 'lng':...",129,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN


In [6]:
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]
clean_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",432,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,4ab2d511f964a5209b6c20e3
1,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,Queen & St. Patrick,43.650505,-79.388577,"[{'label': 'display', 'lat': 43.65050475544005...",400,M5V 1Z1,CA,Toronto,ON,Canada,"[194 Queen St W (Queen & St. Patrick), Toronto...",NaN,4b68aed1f964a520de862be3
2,VFM Test Hotel,Hotel,123 Test Drive,at somewhere St,43.658434,-79.387894,"[{'label': 'display', 'lat': 43.658434, 'lng':...",500,M2M 2M2,CA,Toronto,ON,Canada,"[123 Test Drive (at somewhere St), Toronto ON ...",NaN,4f343a31e4b0230a3b337a90
3,Sheraton Centre Toronto Hotel - Grand Ballroom,Ballroom,123 Queen Street West,at Bay Street,43.651200,-79.384520,"[{'label': 'display', 'lat': 43.65119976208405...",376,NaN,CA,Toronto,ON,Canada,"[123 Queen Street West (at Bay Street), Toront...",Financial District,5545d07e498e2facac03f666
4,Stathcona Hotel,Hotel,NaN,NaN,43.654947,-79.386359,"[{'label': 'display', 'lat': 43.654947, 'lng':...",129,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,4f53fb2ee4b036244bea152f


In [7]:
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2.head()

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
2,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
3,Sheraton Centre Toronto Hotel - Grand Ballroom,Ballroom,123 Queen Street West,43.651200,-79.384520,NaN,ON
4,Stathcona Hotel,Hotel,NaN,43.654947,-79.386359,NaN,ON


In [8]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3.head()

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
2,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
8,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
9,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON


In [9]:
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
8,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
9,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON
10,Hilton,Hotel,145 Richmond St W,43.649946,-79.385479,M5H 2L2,ON
11,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
19,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


In [10]:
# delete rows which has duplicate hotel's name
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
8,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
9,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON
10,Hilton,Hotel,145 Richmond St W,43.649946,-79.385479,M5H 2L2,ON
11,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
19,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


In [11]:
# choose the hotel which has the same postalCode with the event space
df_hotel = df_hotels[df_hotels.postalCode == 'M5H 2M9']
df_hotel

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
19,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


# Create DataFrame PArk

In [12]:
# search for Parks
search_query = 'Park'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=G4G3BSMIJBOKSRFTLMKCCHCOYYAA5B4WMD1AHJPG0YN5RJ5P&client_secret=DSQT5Q4DQVFK04CDBSMY2ALHS5W50FZFVIYTQAZVLXWXKTCR&ll=43.653963,-79.387207&v=20180604&query=Park&radius=10000&limit=30'

In [13]:
presults = requests.get(url).json()
venues = presults['response']['venues']
park_dataframe = json_normalize(venues)
park_dataframe.head()

<ipython-input-13-6d026bc5e16c>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  park_dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4b170e47f964a5208cc123e3,Queen's Park Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1584887051,False,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",716,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",NaN
1,4b54deadf964a520a3d027e3,Grange Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1584887051,False,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",423,NaN,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",NaN
2,4b9d206bf964a520e69136e3,Queen's Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1584887051,False,University Ave.,at Wellesley Ave.,43.663946,-79.392180,"[{'label': 'display', 'lat': 43.66394609897775...",1181,M5R 2E8,CA,Toronto,ON,Canada,"[University Ave. (at Wellesley Ave.), Toronto ...",NaN
3,4d3a1d23325ff04db2dd1845,Residences at College Park North,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",v-1584887051,False,763 Bay St.,btwn. College St. and Gerrard St.,43.659822,-79.385159,"[{'label': 'display', 'lat': 43.65982157793473...",672,M5G 2R3,CA,Toronto,ON,Canada,[763 Bay St. (btwn. College St. and Gerrard St...,NaN
4,4bb79860cf2fc9b616779e02,Berczy Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1584887051,False,35 Wellington St. East,NaN,43.648048,-79.375172,"[{'label': 'display', 'lat': 43.64804835388851...",1171,NaN,CA,Toronto,ON,Canada,"[35 Wellington St. East, Toronto ON, Canada]",NaN


In [14]:
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]
clean_park_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Queen's Park Subway Station,Metro Station,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",716,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",4b170e47f964a5208cc123e3
1,Grange Park,Park,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",423,NaN,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",4b54deadf964a520a3d027e3
2,Queen's Park,Park,University Ave.,at Wellesley Ave.,43.663946,-79.392180,"[{'label': 'display', 'lat': 43.66394609897775...",1181,M5R 2E8,CA,Toronto,ON,Canada,"[University Ave. (at Wellesley Ave.), Toronto ...",4b9d206bf964a520e69136e3
3,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,btwn. College St. and Gerrard St.,43.659822,-79.385159,"[{'label': 'display', 'lat': 43.65982157793473...",672,M5G 2R3,CA,Toronto,ON,Canada,[763 Bay St. (btwn. College St. and Gerrard St...,4d3a1d23325ff04db2dd1845
4,Berczy Park,Park,35 Wellington St. East,NaN,43.648048,-79.375172,"[{'label': 'display', 'lat': 43.64804835388851...",1171,NaN,CA,Toronto,ON,Canada,"[35 Wellington St. East, Toronto ON, Canada]",4bb79860cf2fc9b616779e02


In [15]:
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2.head()

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
1,Grange Park,Park,NaN,43.652488,-79.392053,NaN,ON
2,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
3,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
4,Berczy Park,Park,35 Wellington St. East,43.648048,-79.375172,NaN,ON


In [16]:
# delete rows with none values
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3.head()

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
2,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
3,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
6,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
7,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON


In [17]:

# delete rows which its category is not Park
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park.head()

,name,categories,address,lat,lng,postalCode,state
2,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
10,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
13,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,M5V 3Z1,ON
14,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
17,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON


# Create DataFrame Restaurant

In [18]:
# search for Restaurants
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=G4G3BSMIJBOKSRFTLMKCCHCOYYAA5B4WMD1AHJPG0YN5RJ5P&client_secret=DSQT5Q4DQVFK04CDBSMY2ALHS5W50FZFVIYTQAZVLXWXKTCR&ll=43.653963,-79.387207&v=20180604&query=Restaurant&radius=10000&limit=30'

In [19]:
Rresults = requests.get(url).json()
venues = Rresults['response']['venues']
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

<ipython-input-19-9accf9f220c3>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  Restaurant_dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,4ad4c05ff964a52048f720e3,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1584887053,False,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",145,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,NaN
1,4b223f5af964a520ba4424e3,Azure Restaurant & Bar,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1584887053,False,225 Front St W,43.644749,-79.385113,"[{'label': 'display', 'lat': 43.64474919591934...",1039,M5V 2X3,CA,Toronto,ON,Canada,[225 Front St W (in InterContinental Toronto C...,in InterContinental Toronto Centre,Entertainment District,136175835
2,4ad4c060f964a5207ff720e3,Rol San Restaurant 龍笙棧,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",v-1584887053,False,323 Spadina Ave.,43.654318,-79.398650,"[{'label': 'display', 'lat': 43.65431754076345...",922,M5T 2E9,CA,Toronto,ON,Canada,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...",at D'Arcy St.,Kensington Market,NaN
3,4ad4c05cf964a520dff520e3,360 Restaurant,"[{'id': '4bf58dd8d48988d123941735', 'name': 'W...",v-1584887053,False,301 Front St W,43.642537,-79.387042,"[{'label': 'display', 'lat': 43.64253731714456...",1271,M5V 2T6,CA,Toronto,ON,Canada,"[301 Front St W (301 Front St. W), Toronto ON ...",301 Front St. W,NaN,NaN
4,4b072e9df964a52009f922e3,Sky Dragon Chinese Restaurant 龍翔酒樓,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",v-1584887053,False,280 Spadina Ave.,43.652783,-79.398174,"[{'label': 'display', 'lat': 43.65278331265585...",892,NaN,CA,Toronto,ON,Canada,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...",at Dundas St. W.,NaN,NaN


In [20]:
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]
clean_Restaurant_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",145,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3
1,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,"[{'label': 'display', 'lat': 43.64474919591934...",1039,M5V 2X3,CA,Toronto,ON,Canada,[225 Front St W (in InterContinental Toronto C...,in InterContinental Toronto Centre,Entertainment District,4b223f5af964a520ba4424e3
2,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,"[{'label': 'display', 'lat': 43.65431754076345...",922,M5T 2E9,CA,Toronto,ON,Canada,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...",at D'Arcy St.,Kensington Market,4ad4c060f964a5207ff720e3
3,360 Restaurant,Wine Bar,301 Front St W,43.642537,-79.387042,"[{'label': 'display', 'lat': 43.64253731714456...",1271,M5V 2T6,CA,Toronto,ON,Canada,"[301 Front St W (301 Front St. W), Toronto ON ...",301 Front St. W,NaN,4ad4c05cf964a520dff520e3
4,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,43.652783,-79.398174,"[{'label': 'display', 'lat': 43.65278331265585...",892,NaN,CA,Toronto,ON,Canada,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...",at Dundas St. W.,NaN,4b072e9df964a52009f922e3


In [21]:
#delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Restaurant_dataframe2.head()

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
2,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
3,360 Restaurant,Wine Bar,301 Front St W,43.642537,-79.387042,M5V 2T6,ON
4,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,43.652783,-79.398174,NaN,ON


In [22]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant.head()

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
2,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
3,360 Restaurant,Wine Bar,301 Front St W,43.642537,-79.387042,M5V 2T6,ON
5,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON


# Create DataFrame Cafeteria

In [23]:
# search for Cafeteria
search_query = 'Cafeteria'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=G4G3BSMIJBOKSRFTLMKCCHCOYYAA5B4WMD1AHJPG0YN5RJ5P&client_secret=DSQT5Q4DQVFK04CDBSMY2ALHS5W50FZFVIYTQAZVLXWXKTCR&ll=43.653963,-79.387207&v=20180604&query=Cafeteria&radius=10000&limit=30'

In [24]:
cresults = requests.get(url).json()
venues = cresults['response']['venues']
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

<ipython-input-24-0fbedda48d2f>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  Cafeteria_dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode
0,4f9165bde4b04fef0087e4e2,Cafeteria,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1584887055,False,The Hospital for Sick Children (SickKids),43.657209,-79.386063,"[{'label': 'display', 'lat': 43.65720930549997...",372,CA,Toronto,ON,Canada,"[The Hospital for Sick Children (SickKids), To...",NaN,NaN
1,528e99a211d262edc3708a6f,Stay Cafeteria 慢走,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1584887055,False,388 Spadina Ave,43.655454,-79.399163,"[{'label': 'display', 'lat': 43.65545427446646...",977,CA,Toronto,ON,Canada,"[388 Spadina Ave (at Nassau St), Toronto ON M5...",at Nassau St,M5T 2G5
2,53bd6dfe498e6c1d3bc9333e,Industrial Alliance Cafeteria,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1584887055,False,NaN,43.655657,-79.389443,"[{'label': 'display', 'lat': 43.655657, 'lng':...",260,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN
3,556f826b498e07f469e6bf1f,Dears Cafeteria,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",v-1584887055,False,NaN,43.654179,-79.392555,"[{'label': 'display', 'lat': 43.65417861938476...",431,CA,NaN,NaN,Canada,[Canada],NaN,NaN
4,4ec2b68a8231a83de8c45191,OCAD Cafeteria,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",v-1584887055,False,100 McCaul St.,43.652780,-79.391762,"[{'label': 'display', 'lat': 43.65278016892648...",389,CA,Toronto,ON,Canada,"[100 McCaul St. (Second Floor), Toronto ON, Ca...",Second Floor,NaN


In [25]:
# keep only columns that include venue name, and anything that is associated with location
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]
clean_Cafeteria_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,id
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,"[{'label': 'display', 'lat': 43.65720930549997...",372,CA,Toronto,ON,Canada,"[The Hospital for Sick Children (SickKids), To...",NaN,NaN,4f9165bde4b04fef0087e4e2
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,"[{'label': 'display', 'lat': 43.65545427446646...",977,CA,Toronto,ON,Canada,"[388 Spadina Ave (at Nassau St), Toronto ON M5...",at Nassau St,M5T 2G5,528e99a211d262edc3708a6f
2,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,"[{'label': 'display', 'lat': 43.655657, 'lng':...",260,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,53bd6dfe498e6c1d3bc9333e
3,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,"[{'label': 'display', 'lat': 43.65417861938476...",431,CA,NaN,NaN,Canada,[Canada],NaN,NaN,556f826b498e07f469e6bf1f
4,OCAD Cafeteria,College Cafeteria,100 McCaul St.,43.652780,-79.391762,"[{'label': 'display', 'lat': 43.65278016892648...",389,CA,Toronto,ON,Canada,"[100 McCaul St. (Second Floor), Toronto ON, Ca...",Second Floor,NaN,4ec2b68a8231a83de8c45191


In [26]:

# delete unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2.head()

,name,categories,address,lat,lng,state,postalCode
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,ON,NaN
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,ON,M5T 2G5
2,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,ON,NaN
3,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,NaN,NaN
4,OCAD Cafeteria,College Cafeteria,100 McCaul St.,43.652780,-79.391762,ON,NaN


In [27]:
# delete rows with none values
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria

,name,categories,address,lat,lng,state,postalCode
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,ON,M5T 2G5
7,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,ON,M4V 1H7
20,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,ON,M5B 2K3
22,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,ON,M5B 1W8
25,Innis College,Student Center,2 Sussex Ave,43.665556,-79.399298,ON,M5S 1J5
28,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,ON,m3c 3k7


# Create DataFrame Shopping

In [28]:
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=G4G3BSMIJBOKSRFTLMKCCHCOYYAA5B4WMD1AHJPG0YN5RJ5P&client_secret=DSQT5Q4DQVFK04CDBSMY2ALHS5W50FZFVIYTQAZVLXWXKTCR&ll=43.653963,-79.387207&v=20180604&query=Shopping&radius=1000&limit=30'

In [29]:
sresults = requests.get(url).json()
venues = sresults['response']['venues']
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

<ipython-input-29-fc82f912e8a4>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  Shopping_dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,4ad4c062f964a520fcf720e3,Dragon City Shopping Mall 龍城,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1584887056,False,280 Spadina Ave,at Dundas St W,43.652774,-79.398222,"[{'label': 'display', 'lat': 43.65277367198862...",897,M5T 3A5,CA,Toronto,ON,Canada,"[280 Spadina Ave (at Dundas St W), Toronto ON ...",NaN
1,4ad77a12f964a520260b21e3,CF Toronto Eaton Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1584887056,False,220 Yonge St,btwn Queen & Dundas,43.654540,-79.380677,NaN,529,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...","Downtown Toronto, Toronto, ON"
2,4b9e86dff964a520e6eb36e3,TD Centre Shopping Concourse,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1584887056,False,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,"[{'label': 'display', 'lat': 43.64718424611984...",908,M5K 1A1,CA,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...",NaN
3,5840cafe06f1a34af80cc609,Saks Fifth Avenue Club - Personal Shopping,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",v-1584887056,False,176 Yonge Street,Queen Street West,43.651810,-79.379192,"[{'label': 'display', 'lat': 43.65181, 'lng': ...",688,M5C 2L7,CA,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...",NaN
4,4b83d653f964a520d91231e3,Yellow Pages - RedFlagDeals.com,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1584887056,False,461 King St. West,NaN,43.647958,-79.396006,"[{'label': 'display', 'lat': 43.64795765580337...",974,M5V 1K4,CA,Toronto,ON,Canada,"[461 King St. West, Toronto ON M5V 1K4, Canada]",NaN


In [30]:
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]
clean_Shopping_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,at Dundas St W,43.652774,-79.398222,"[{'label': 'display', 'lat': 43.65277367198862...",897,M5T 3A5,CA,Toronto,ON,Canada,"[280 Spadina Ave (at Dundas St W), Toronto ON ...",NaN,4ad4c062f964a520fcf720e3
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,btwn Queen & Dundas,43.654540,-79.380677,NaN,529,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...","Downtown Toronto, Toronto, ON",4ad77a12f964a520260b21e3
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,"[{'label': 'display', 'lat': 43.64718424611984...",908,M5K 1A1,CA,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...",NaN,4b9e86dff964a520e6eb36e3
3,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,Queen Street West,43.651810,-79.379192,"[{'label': 'display', 'lat': 43.65181, 'lng': ...",688,M5C 2L7,CA,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...",NaN,5840cafe06f1a34af80cc609
4,Yellow Pages - RedFlagDeals.com,Office,461 King St. West,NaN,43.647958,-79.396006,"[{'label': 'display', 'lat': 43.64795765580337...",974,M5V 1K4,CA,Toronto,ON,Canada,"[461 King St. West, Toronto ON M5V 1K4, Canada]",NaN,4b83d653f964a520d91231e3


In [31]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,43.652774,-79.398222,M5T 3A5,ON
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
3,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
4,Yellow Pages - RedFlagDeals.com,Office,461 King St. West,43.647958,-79.396006,M5V 1K4,ON


In [32]:
# delete rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping

,name,categories,address,lat,lng,postalCode,state
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,43.652774,-79.398222,M5T 3A5,ON
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON


# map to visualize hotels, shopping stores and Cafeteria and how they cluster together¶

In [33]:
# create dataframe of hotels, shopping stores and Cafeteria
hotel_neighbourhood_df = pd.concat([df_hotel, df_Cafeteria, df_Shopping], ignore_index=True)
hotel_neighbourhood_df.head()

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON
2,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
3,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
4,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON


In [34]:
# Generate map to visualize hotel neighbourhood including shopping stores and Cafeteria 
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

# map to visualize Park, Restaurant and Cafeteria and how they cluster together

In [38]:
# create dataframe of Park, Restaurant and Cafeteria
park_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
park_neighbourhood_df.head()

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
1,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
2,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,M5V 3Z1,ON
3,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
4,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON


In [36]:
# Generate map to visualize park neighbourhood including Restaurant and Cafeteria 
park_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(park_map)  
    
park_map